In [3]:
using JuMP, GLPK, Plots

In [42]:
m = Model(GLPK.Optimizer)
@variable(m, 0 <= x, Int)
@variable(m, 0 <= y, Int)
@constraint(m, x + y <= 6)
@constraint(m, 5x + 9y <= 45)

@objective(m, Max, 5x+8y)

print(m)

In [43]:
status = optimize!(m)

In [44]:
value(x), value(y)

(0.0, 5.0)

In [62]:
m = Model(GLPK.Optimizer)
@variable(m, 0 <= x)
@variable(m, 0 <= y)
@constraint(m, x + y <= 6)
@constraint(m, 5x + 9y <= 45)

@objective(m, Max, 5x+8y)

print(m)

In [63]:
status = optimize!(m)

In [64]:
value(x), value(y)

(2.25, 3.75)

In [65]:
function pivot!(M::Matrix, i::Int, j::Int)
    @assert M[i, j] != 0 "pivot sur coefficient nul"
    M[i, :] /= M[i, j]
    m, n = size(M)
    for k in 1:m
        if i != k
            coef = M[k, j]
            M[k, :] -= coef * M[i, :]
        end
    end
    return M
end

pivot! (generic function with 1 method)

In [66]:
A = [ 1.0 1 1 0; 5 9 0 1 ]
b = [ 6 ; 45 ]
c = [ -5 -8 0 0 0 ]
M = [ A b ; c ]

3×5 Matrix{Float64}:
  1.0   1.0  1.0  0.0   6.0
  5.0   9.0  0.0  1.0  45.0
 -5.0  -8.0  0.0  0.0   0.0

In [67]:
pivot!(M,2,2)

3×5 Matrix{Float64}:
  0.444444  0.0  1.0  -0.111111   1.0
  0.555556  1.0  0.0   0.111111   5.0
 -0.555556  0.0  0.0   0.888889  40.0

In [68]:
-5/9

-0.5555555555555556

In [69]:
pivot!(M,1,1)

3×5 Matrix{Float64}:
 1.0  0.0   2.25  -0.25   2.25
 0.0  1.0  -1.25   0.25   3.75
 0.0  0.0   1.25   0.75  41.25

Partons de la première contrainte pour former la coupe
$$
-0.25s_1 - 0.75s_2 \leq -0.25
$$

In [70]:
(m,n) = size(M)
M = hcat(M, Matrix{Rational{Int64}}(zeros(m,1)))
M[1:m,n+1] = M[1:m,n]
M[1:m,n] = zeros(m,1)
M = vcat(M, Matrix{Rational{Int64}}(zeros(1,n+1)))
M[m+1,:] = M[m,:]
M[m,:] = [ 0 0 -1//4 -3//4 1 -1//4]
M

4×6 Matrix{Float64}:
 1.0  0.0   2.25  -0.25  0.0   2.25
 0.0  1.0  -1.25   0.25  0.0   3.75
 0.0  0.0  -0.25  -0.75  1.0  -0.25
 0.0  0.0   1.25   0.75  0.0  41.25

In [71]:
pivot!(M, 3, 4)

4×6 Matrix{Float64}:
  1.0   0.0   2.33333   0.0  -0.333333   2.33333
  0.0   1.0  -1.33333   0.0   0.333333   3.66667
 -0.0  -0.0   0.333333  1.0  -1.33333    0.333333
  0.0   0.0   1.0       0.0   1.0       41.0

In [72]:
(m,n) = size(M)
M = hcat(M, Matrix{Rational{Int64}}(zeros(m,1)))
M[1:m,n+1] = M[1:m,n]
M[1:m,n] = zeros(m,1)
M = vcat(M, Matrix{Rational{Int64}}(zeros(1,n+1)))
M[m+1,:] = M[m,:]
M[m,:] = [ 0 0 -1//3 0 2//3 0 -1//3]
M

5×7 Matrix{Float64}:
  1.0   0.0   2.33333   0.0  -0.333333  0.0   2.33333
  0.0   1.0  -1.33333   0.0   0.333333  0.0   3.66667
 -0.0  -0.0   0.333333  1.0  -1.33333   0.0   0.333333
  0.0   0.0  -0.333333  0.0   0.666667  0.0  -0.333333
  0.0   0.0   1.0       0.0   1.0       0.0  41.0

In [73]:
pivot!(M, 4, 3)

5×7 Matrix{Float64}:
  1.0   0.0  0.0   0.0   4.33333    0.0   0.0
  0.0   1.0  0.0   0.0  -2.33333    0.0   5.0
  0.0   0.0  0.0   1.0  -0.666667   0.0   0.0
 -0.0  -0.0  1.0  -0.0  -2.0       -0.0   1.0
  0.0   0.0  0.0   0.0   3.0        0.0  40.0

In [74]:
m = Model(GLPK.Optimizer)
@variable(m, 0 <= x)
@variable(m, 0 <= y)
@constraint(m, x + y <= 6)
@constraint(m, 5x + 9y <= 45)

@objective(m, Max, 5x+8y)

A = [ 1.0 1 1 0; 5 9 0 1 ]
b = [ 6 ; 45 ]
c = [ -5 -8 0 0 0 ]
M = [ A b ; c ]

pivot!(M,2,2)
pivot!(M,1,1)

3×5 Matrix{Float64}:
 1.0  0.0   2.25  -0.25   2.25
 0.0  1.0  -1.25   0.25   3.75
 0.0  0.0   1.25   0.75  41.25

In [ ]:
plot(x -> (18 - 3x)/2, fill = (-2, 0.2, :red), color = :red, 
    linewidth = 2, label = "contrainte 1")
plot!(xlims = (-0.5,  4.5), ylims = (-0.5, 6.5))
hline!([0,  6], color = :black, linestyle = :dash, linewidth = 2, label = "")
vline!([0, 4], color = :black, linestyle = :dash, linewidth = 2, label = "")
plot!(title = "Je vis et je respire Julia")
plot!(xlabel = "x")
plot!(ylabel = "y")
scatter!([0, 0, 2, 4, 4], [0, 6, 6, 0, 3], markersize = 7, color = :blue, label = "Points extrêmes")